## Import Modules

In [1]:
import dataset as ds
import config as cfg
import tools
seed_time = tools.generate_seed()
from evaluate import Evaluate
import torch
from ultralytics import settings
from ultralytics import YOLO

from sklearn.model_selection import train_test_split


from clearml import Task
import clearml
clearml.browser_login()


# ClearML
https://app.clear.ml/dashboard

# Yolo Config

In [2]:
# Ultralytics settings
settings.reset()
settings.update({'datasets_dir': cfg.YOLO_DATASET_DIR.replace('/','\\')})
print(settings)

# GPU Utilisation
print(torch.cuda.is_available())
device = torch.device("cuda")
print(torch.backends.cudnn.version())


{'settings_version': '0.0.4', 'datasets_dir': 'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets\\YOLOv8', 'weights_dir': 'weights', 'runs_dir': 'runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
True
8801


## Load Dataset

In [3]:
# Base paths for the images and labels
train_images_path = f'{cfg.BASE_DATASET}/images/train'
train_labels_path = f'{cfg.BASE_DATASET}/labels/train'
val_images_path = f'{cfg.BASE_DATASET}/images/valid'
val_labels_path = f'{cfg.BASE_DATASET}/labels/valid'

# Base path for metadata
metadata_path = f'{cfg.BASE_DATASET}/metadata.json'

# Create the DataFrames for the train and validation sets
train_df = ds.create_dataframe(train_images_path, train_labels_path, metadata_path)
valid_df = ds.create_dataframe(val_images_path, val_labels_path, metadata_path)

# always train on Palo Alto for Consitency
train_df = train_df[(train_df['location'] == 'Palo Alto')]
valid_df = valid_df[(valid_df['location'] == 'Palo Alto')]


# Test Parameters

In [4]:
# name dataset to be create as well as task name
dataset_name = 'test500'

# project and task constant parameters
#project_name= cfg.YOLO_PROJECT_NAME
dataset_dir = f'{cfg.YOLO_DATASET_DIR}/{dataset_name}'
project_dir = f'{cfg.YOLO_PROJECT_DIR}/{dataset_name}/'
class_names = cfg.YOLO_CLASS_NAMES

# task specifc training parameters
epochs = 30
model_variant = "yolov8n"
RUN = 1

train_size = 500
val_size = int(train_size * 0.25)
w1 = 0
w2 = 1


task_name = 'r5'
task_name = f'{task_name}-w{w1}{w2}-{epochs}-{train_size}-{RUN}'
project_name= cfg.YOLO_PROJECT_NAME + f'/automation-test'


augmentation_metadata = {
        'methods': {        
            'rotate': {
                'parameters': {
                    'angle_limit': 5, # Maximum angle of rotation
                    'p': 1.0  # Probability of applying the augmentation
                },
                'apply_to_percentage': 0.5  # 50% of the training images
            }        
        }
    }


# Create Datasets

In [5]:

# fitler by weather
filtered_train_df = train_df[(train_df['weather'] == w1) | (train_df['weather'] == w2)]
filtered_valid_df = valid_df[(valid_df['weather'] == w1) | (valid_df['weather'] == w2)]

# Create a combined stratification key
filtered_train_df['stratify_key'] = filtered_train_df['ac'] + '_' + filtered_train_df['weather'].astype(str)
filtered_valid_df['stratify_key'] = filtered_valid_df['ac'] + '_' + filtered_valid_df['weather'].astype(str)

_, test_train_df = train_test_split(
    filtered_train_df,
    test_size=train_size,  # Number of items you want in your sample
    stratify=filtered_train_df['stratify_key'],  # Stratify based on the combined column
    random_state=seed_time  # Ensures reproducibility
)

_, test_val_df = train_test_split(
    filtered_valid_df,
    test_size=val_size,  # Number of items you want in your sample
    stratify=filtered_valid_df['stratify_key'],  # Stratify based on the combined column
    random_state=seed_time  # Ensures reproducibility
)

# create sub dataset
ds.create_sub_dataset(dataset_dir, test_train_df, test_val_df, class_names)

# correct dataset labels
ds.correct_dataset_labels(dataset_dir, test_train_df, test_val_df, class_names)

# augment dataset
ds.augment_dataset(dataset_dir, augmentation_metadata)

C:\Users\tomev\AppData\Local\Temp\ipykernel_32400\3253271123.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df['stratify_key'] = filtered_train_df['ac'] + '_' + filtered_train_df['weather'].astype(str)
C:\Users\tomev\AppData\Local\Temp\ipykernel_32400\3253271123.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_valid_df['stratify_key'] = filtered_valid_df['ac'] + '_' + filtered_valid_df['weather'].astype(str)


Removing dataset if pre-existing
Copying training files:


Copying files: 100%|██████████| 1000/1000 [00:00<00:00, 1399.19it/s]


Copying validation files:


Copying files: 100%|██████████| 250/250 [00:00<00:00, 1312.00it/s]


...

Processing valid labels in test500:


Processing labels: 100%|██████████| 125/125 [00:00<00:00, 334.60it/s]


Label correction completed.


Applying augmentations: 100%|██████████| 250/250 [00:03<00:00, 82.75it/s]


An error occurred: Expected x_max for bbox (0.139949, 0.11087449999999999, 1.0701450000000001, 0.7310055, 2) to be in the range [0.0, 1.0], got 1.0701450000000001.
An error occurred: Expected x_max for bbox (0.139949, 0.11087449999999999, 1.0701450000000001, 0.7310055, 2) to be in the range [0.0, 1.0], got 1.0701450000000001.


# Training

In [6]:
# train on pure dataset

# dataset location
dataset_path=f'{dataset_dir}\\{dataset_name}.yaml'
project =  project_dir + 'pure' #save_dir # weight save path

# Create ClearML task
task_pure = Task.init(
    project_name=project_name,
    task_name=f"{task_name}-pure"
)
task_pure.set_parameter("model_variant", model_variant)

# Define Yolo model
#model = YOLO(f'{model_variant}.yaml') # train on model which is not pre-trained
model_pure = YOLO(f'{model_variant}.pt')

#train args
args_pure = dict(data=dataset_path, 
            epochs=epochs, 
            device=0, 
            #close_mosaic=epochs, # disable mosaic augmentation
            seed=42
            )
task_pure.connect(args_pure)

# train model
results_pure=model_pure.train(**args_pure, project=project)

# validate model
metrics_pure = model_pure.val()
metrics_dict_pure = Evaluate.get_yolo_metrics_dict(metrics_pure)

# Log the metrics to ClearML
task_pure.upload_artifact('VAL_METRICS', tools.pretty_print_dict(metrics_dict_pure))

print("Closing Task Pure ...")
task_pure.close()
print("done")

ClearML Task: created new task id=9346d1202f7e441183367cbf60b4f4ec
2024-05-31 23:34:05,716 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/e51413769c444f7788933dc4df4fad17/experiments/9346d1202f7e441183367cbf60b4f4ec/output/log
2024-05-31 23:34:11,299 - clearml.model - INFO - Selected model id: 6932d54ba60d49f7bd3432493468f8c1
New https://pypi.org/project/ultralytics/8.2.27 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.10  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets/YOLOv8/test500\test500.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/logs/YOLOv8/test500/pure, name=train3, exist_ok=False, p

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\train... 500 images, 0 backgrounds, 1 corrupt: 100%|██████████| 500/500 [00:00<00:00, 1557.20it/s]

train: WARNING  C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\images\train\48653.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0289]
train: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\valid... 125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 125/125 [00:00<00:00, 365.91it/s]

...


30 epochs completed in 0.058 hours.
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train3\weights\last.pt, 6.2MB
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train3\weights\best.pt, 6.2MB

Validating C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train3\weights\best.pt...
Ultralytics YOLOv8.2.10  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.25it/s]


                   all        125        125      0.593      0.519      0.591       0.32
        Cessna Skyhawk        125         42      0.548      0.375      0.478      0.223
        Boeing 737-800        125         41      0.644      0.683      0.709      0.407
          King Air C90        125         42      0.588        0.5      0.586      0.329
Speed: 0.3ms preprocess, 1.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train3
2024-05-31 23:38:50,582 - clearml.storage - INFO - Starting upload: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train3\weights\best.pt => https://files.clear.ml/Intruder-Aircraft-Detection/YOLOv8/automation-test/r5-w01-30-500-1-pure.9346d1202f7e441183367cbf60b4f4ec/models/best.pt
Ultralytics YOLOv8.2.10  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers,

val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\valid.cache... 125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 125/125 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  88%|████████▊ | 7/8 [00:07<00:00,  2.41it/s]c:\Users\tomev\anaconda3\envs\yolo\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning:

Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]


2024-05-31 23:38:59,753 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Intruder-Aircraft-Detection/YOLOv8/automation-test/r5-w01-30-500-1-pure.9346d1202f7e441183367cbf60b4f4ec/models/best.pt
                   all        125        125      0.594      0.521      0.593      0.319
        Cessna Skyhawk        125         42      0.552      0.381      0.483      0.228
        Boeing 737-800        125         41      0.643      0.683      0.708        0.4
          King Air C90        125         42      0.587        0.5      0.586      0.329
Speed: 0.8ms preprocess, 5.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train32
Closing Task Pure ...
done


In [7]:
# train on augmented dataset

# clear variables

# dataset location
dataset_path=f'{dataset_dir}\\{dataset_name}-aug.yaml'
project =  project_dir + 'augmented' #save_dir # weight save path

# Create ClearML task
task_aug = Task.init(
    project_name=project_name,
    task_name=f"{task_name}-aug"
)
task_aug.set_parameter("model_variant", model_variant) 

# Define Yolo model
#model = YOLO('yolov8n.yaml')
model_aug = YOLO('yolov8n.pt')

# train model
results_aug = model_aug.train(data=dataset_path, 
                      epochs=epochs, 
                      device=0, 
                      project=project,
                      #close_mosaic=epochs, # disable mosaic augmentation
                      seed=42)

# validate model
metrics_aug = model_aug.val()
metrics_dict_aug = Evaluate.get_yolo_metrics_dict(metrics_aug)

# Log the metrics to ClearML
task_aug.upload_artifact('VAL_METRICS', tools.pretty_print_dict(metrics_dict_aug))
task_aug.upload_artifact('AUGMENTATION_METADATA', tools.pretty_print_dict(augmentation_metadata))

print("Closing Task Augmented ...")
task_aug.close()

print("done")

ClearML Task: created new task id=ee3dee63bad24a9fa20afaa117753ddf
ClearML results page: https://app.clear.ml/projects/e51413769c444f7788933dc4df4fad17/experiments/ee3dee63bad24a9fa20afaa117753ddf/output/log


Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely


New https://pypi.org/project/ultralytics/8.2.27 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.10  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets/YOLOv8/test500\test500-aug.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/logs/YOLOv8/test500/augmented, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True,

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\train-aug... 748 images, 0 backgrounds, 1 corrupt: 100%|██████████| 748/748 [00:00<00:00, 840.22it/s]

train: WARNING  C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\images\train-aug\48653.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0289]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.47it/s]


                   all        125        125      0.679      0.568      0.655      0.384
        Cessna Skyhawk        125         42      0.749      0.426       0.52       0.26
        Boeing 737-800        125         41      0.753      0.594      0.728      0.452
          King Air C90        125         42      0.534      0.684      0.718      0.439
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\augmented\train
Ultralytics YOLOv8.2.10  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\valid.cache... 125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 125/125 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]


                   all        125        125      0.738      0.533      0.655      0.381
        Cessna Skyhawk        125         42      0.734      0.381      0.522      0.266
        Boeing 737-800        125         41      0.883      0.552      0.726      0.445
          King Air C90        125         42      0.596      0.667      0.717      0.432
Speed: 0.4ms preprocess, 3.3ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\augmented\train2
Closing Task Augmented ...
done


: 